In [12]:
import os
import cv2
import pycolmap
from hloc.utils import viz_3d
import numpy as np
import pixtrack.utils.pose_utils as pose_utils
from pixloc.utils.colmap import Camera as ColCamera
from pixloc.pixlib.geometry import Camera as PixCamera, Pose
import matplotlib.pyplot as plt
from pixtrack.utils.ingp_utils import load_nerf2sfm, initialize_ingp, sfm_to_nerf_pose, nerf_to_sfm_pose
from pixtrack.visualization.run_vis_on_poses import get_nerf_image
from scipy.spatial.transform import Rotation as R

In [13]:
sfm_path = '/data/pixtrack/outputs/nerf_sfm/aug_gimble_04MAR2022/aug_sfm/'
model = pycolmap.Reconstruction(sfm_path)


nerf_path = '/data/pixtrack/instant-ngp/snapshots/gimble_04MAR2022/weights.msgpack'
nerf2sfm_path = '/data/pixtrack/pixel-perfect-sfm/datasets/gimble_04MAR2022/nerf2sfm.pkl'
nerf2sfm = load_nerf2sfm(nerf2sfm_path)
testbed = initialize_ingp(nerf_path)

02:57:42 INFO     Loading network config from: /data/pixtrack/instant-ngp/snapshots/gimble_04MAR2022/weights.msgpack
02:57:42 INFO     GridEncoding:  Nmin=16 b=1.44727 F=2 T=2^19 L=16
02:57:42 INFO     Density model: 3--[HashGrid]-->32--[FullyFusedMLP(neurons=64,layers=3)]-->1
02:57:42 INFO     Color model:   3--[Composite]-->16+16--[FullyFusedMLP(neurons=64,layers=4)]-->3
02:57:42 INFO       total_encoding_params=12599920 total_network_params=10240


In [14]:
camera = model.cameras[1]
camera = ColCamera(None, 
                camera.model_name,
                int(camera.width),
                int(camera.height),
                camera.params)
camera = PixCamera.from_colmap(camera)

In [15]:
all_imgs_in_altitude = []
for i in range(33, 73):
    all_imgs_in_altitude.append(f"mapping/IMG_95{i}.png")
    
render_poses = []
for image_name in all_imgs_in_altitude:
    image = model.find_image_with_name(image_name)
    cIw = pose_utils.get_camera_in_world_from_colmap_image(image)
    render_poses.append(sfm_to_nerf_pose(nerf2sfm, cIw))



In [16]:
!pip3 install mediapy

In [18]:
from scipy.spatial.transform import Rotation as R
from scipy.spatial.transform import Slerp

In [ ]:
image_renders = []
intermediate_frames = 20
for pose_number in range(len(render_poses)):
    pose1 = render_poses[pose_number]
    if pose_number+1<len(render_poses):
        pose2 = render_poses[pose_number+1]
        rotations = np.zeros((2, 3, 3))
        rotations[0, :, :] = pose1[:3, :3]
        rotations[1, :, :] = pose2[:3, :3]
        translations_x = np.linspace(pose1[0, -1], pose2[0, -1], intermediate_frames)
        translations_y = np.linspace(pose1[1, -1], pose2[1, -1], intermediate_frames)
        translations_z = np.linspace(pose1[2, -1], pose2[2, -1], intermediate_frames)
    else:
        break

    interpolation_times = np.arange(0, intermediate_frames).tolist()
    key_times = [0, intermediate_frames-1]    
    key_rotations = R.from_matrix(rotations)
    slerp = Slerp(key_times, key_rotations)
    intermediate_rotations = slerp(interpolation_times).as_matrix()
    #print(translations_x.shape[0])
    for pose_num in range(translations_x.shape[0]):
        camera_pose = np.eye(4)
        camera_pose[0, -1] = translations_x[pose_num]
        camera_pose[1, -1] = translations_y[pose_num]
        camera_pose[2, -1] = translations_z[pose_num]
        camera_pose[:3, :3] = intermediate_rotations[pose_num, :, :]
        nerf_img = get_nerf_image(testbed,camera_pose, camera)
        image_renders.append(nerf_img)


In [ ]:
mediapy.show_video(image_renders, width=320, height=240, fps=10)